In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install flask flask-ngrok torch transformers diffusers peft ngrok flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 363.4/363.4 MB 239.8 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
import io
import uuid
import subprocess

from flask import Flask, request, send_file, abort
from flask_cors import CORS

import torch
from PIL import Image
from diffusers import (
    StableDiffusionImg2ImgPipeline,
    DPMSolverMultistepScheduler,
    AutoencoderKL,
    UNet2DConditionModel
)
from huggingface_hub import login

# 1. Install dependencies (if needed)
def install_dependencies():
    subprocess.run(
        ['pip', 'install', 'pyngrok', 'flask', 'torch', 'diffusers', 'transformers',
         'accelerate', 'peft', 'pillow'],
        check=True
    )

install_dependencies()

# 2. Flask app setup
app = Flask(__name__)
CORS(app)

# 3. Ngrok setup (optional)
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("")  # Replace with your actual token
public_url = ngrok.connect(5051).public_url
print(f" * Ngrok tunnel: {public_url}")

# 4. Initialize the pipeline with SDXL
pipe = None
def initialize_pipeline():
    global pipe
    login(token="YOUR_HUGGINGFACE_TOKEN")  # Replace with your Hugging Face token

    # Use SDXL base model with proper pipeline class
    pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",  # Base model instead of refiner
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    ).to("cuda")

    pipe.enable_attention_slicing()

initialize_pipeline()

# 5. Modified /generate endpoint
@app.route('/generate', methods=['POST'])
def generate():
    if 'prompt' not in request.form or 'image' not in request.files:
        abort(400, description="Missing 'prompt' or 'image'")

    prompt = request.form['prompt']
    init_file = request.files['image']

    try:
        init_image = Image.open(init_file).convert("RGB").resize((1024, 1024))
    except Exception as e:
        abort(400, description=f"Image error: {e}")

    output = pipe(
        prompt=prompt,
        image=init_image,
        strength=0.7,
        guidance_scale=7.5,
        num_inference_steps=50
    )

    img_io = io.BytesIO()
    output.images[0].save(img_io, format='PNG')
    img_io.seek(0)
    return send_file(img_io, mimetype='image/png')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5051)

ModuleNotFoundError: No module named 'flask_cors'